<a href="https://colab.research.google.com/github/Danymatty08/huggingface-dataset/blob/main/sentiment_analysis_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load dataset

Create a secret variable named "gh_username" with your github/huggingface username

In [1]:
from google.colab import userdata
username = userdata.get('gh_username')
print(username)

Smatteux


In [4]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [9]:
from datasets import load_dataset

# Change the username with yours
dataset = load_dataset(f"{username}/sentiment-analysis-test")


# Tokenize

In [10]:
# Add label column to dataset

label2id = {"negative": 0, "neutral": 1, "positive": 2}

def add_label_column(examples):
    examples["label"] = label2id[examples["sentiment"]]
    return examples

dataset = dataset.map(add_label_column)
print(dataset)


Map:   0%|          | 0/279 [00:00<?, ? examples/s]

Map:   0%|          | 0/70 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'sentiment', 'label'],
        num_rows: 279
    })
    test: Dataset({
        features: ['text', 'sentiment', 'label'],
        num_rows: 70
    })
})


In [12]:
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/xlm-roberta-base-tweet-sentiment-it")

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
print(tokenized_dataset)


tokenizer_config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Map:   0%|          | 0/279 [00:00<?, ? examples/s]

Map:   0%|          | 0/70 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'sentiment', 'label', 'input_ids', 'attention_mask'],
        num_rows: 279
    })
    test: Dataset({
        features: ['text', 'sentiment', 'label', 'input_ids', 'attention_mask'],
        num_rows: 70
    })
})


In [14]:
print(tokenized_dataset["train"][0])
print(tokenized_dataset["test"][0])

{'text': 'L’entusiasmo dei ragazzi nell’apprendimento è nell’affrontare nuove sfide ', 'sentiment': 'positive', 'label': 2, 'input_ids': [0, 339, 26, 2517, 86840, 432, 1613, 58202, 10076, 26, 214206, 565, 10076, 26, 193785, 1046, 41809, 61875, 112, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'text': 'Sono importanti perché utili alle competenze lavorative necessarie', 'sentiment': 'positive', 'label': 2, 'input_ids': [0, 23992, 54880, 10724, 19537, 747, 128310, 39335, 4935, 43216, 13, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


# Fine tuning a pre-trained model

In [20]:
from transformers import AutoModelForSequenceClassification

checkpoint = "cardiffnlp/xlm-roberta-base-tweet-sentiment-it"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

In [21]:
print(model.config)

XLMRobertaConfig {
  "_attn_implementation_autoset": true,
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "negative",
    "1": "neutral",
    "2": "positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 0,
    "neutral": 1,
    "positive": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.51.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}



In [22]:
# Create training parameters

from transformers import TrainingArguments

training_args = TrainingArguments("sentiment-analysis-test")



In [23]:
# Create trainer

from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator
)

In [28]:
# Train the model

trainer.train()

Step,Training Loss


KeyboardInterrupt: 

In [ ]:
# Push the new model to the hub
trainer.push_to_hub()


# Evaluate!

In [ ]:
from transformers import pipeline, AutoModelForSequenceClassification

# Load and test the original model
sentiment_pipeline_orig = pipeline("sentiment-analysis", model="cardiffnlp/xlm-roberta-base-tweet-sentiment-it")

prompt = "Personale docente"
print(f"Sentiment for prompt {prompt}:")
print(sentiment_pipeline_orig(prompt))


prompt = "Interrogazioni a sorpresa"
print(f"Sentiment for prompt {prompt}:")
print(sentiment_pipeline_orig(prompt))

In [ ]:
from transformers import pipeline, AutoModelForSequenceClassification

# Load and test our model
sentiment_pipeline = pipeline("sentiment-analysis", model=f"{username}/sentiment-analysis-test")

prompt = "Personale docente"

print(f"Sentiment for prompt {prompt}:")
print(sentiment_pipeline(prompt))

prompt = "Interrogazioni a sorpresa"
print(f"Sentiment for prompt {prompt}:")
print(sentiment_pipeline(prompt))

# Gradio interface

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

# Load and test our model
sentiment_pipeline = pipeline("sentiment-analysis", model=f"{username}/sentiment-analysis-test")

def analyze_sentiment(text):
    result = sentiment_pipeline(text)
    label = result[0]['label']
    score = result[0]['score']
    return f"Label: {label}, Score: {score}"

iface = gr.Interface(
    fn=analyze_sentiment,
    inputs=gr.Textbox(label="Prompt", lines=2, placeholder="Scrivi qui qualcosa sulla tua scuola..."),
    outputs=gr.Textbox(label="Sentiment Analysis Result"),
    title="Sentiment Analysis for VIAN",
    description="Analizza i sentimenti riguardo alla tua scuola con un modello fine-tuned",
)

iface.launch()
